# Custom User-based Model
The present notebooks aims at creating a UserBased class that inherits from the Algobase class (surprise package) and that can be customized with various similarity metrics, peer groups and score aggregation functions. 

In [70]:
# reloads modules automatically before entering the execution of code
%load_ext autoreload
%autoreload 2

# standard library imports
# -- add new imports here --

# third parties imports
import numpy as np 
import pandas as pd
from surprise import AlgoBase
# -- add new imports here --

# local imports
from constants import Constant as C
from loaders import load_ratings
# -- add new imports here --
from surprise import Dataset, KNNWithMeans



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Loading Data
Prepare a dataset in order to help implementing a user-based recommender system

In [71]:
# -- load data, build trainset and anti testset --
ratings = load_ratings(surprise_format=True)
trainset = ratings.build_full_trainset()
anti_testset = trainset.build_anti_testset()

# 2. Explore Surprise's user-based algorithm
Displays user-based predictions and similarity matrix on the test dataset using the KNNWithMeans class

In [72]:
sim_options = {
    'name': 'msd',  # Mean Squared Difference
    'user_based': True,  # Modèle basé sur les utilisateurs
    'min_support': 3  # Minimum de 3 évaluations communes
}
# Créer une instance de KNNWithMeans
algo = KNNWithMeans(k=3, min_k=2, sim_options=sim_options)
# Entraîner le modèle
algo.fit(trainset)
# Faire une prédiction pour l'utilisateur 11 et l'élément 364
prediction = algo.predict(uid=11, iid=364)
print(f"Prédiction pour l'utilisateur 11 et l'élément 364 : {prediction.est}")


predictions = algo.test(anti_testset)
for pred in predictions[:30]:
    print(f"Utilisateur {pred.uid} a évalué l'élément {pred.iid} avec une note de {pred.est:.2f} (réel : {pred.r_ui})")
#1.La valeur de min_support est fixée à 3, ce qui signifie que pour qu'un voisin soit pris en compte dans le calcul de la prédiction il doit avoir au moins 3 évaluations communes avec l'utilisateur cible. Cela peut réduire le nombre de voisins valides et donc influencer la prédiction finale.
#2.Quand min_support est fixé à 3, la valeur de actual_k diminue pour certaines prédictions. Cela est dû au fait que actual_k représente le nombre de voisins qui ont réellement été utilisés pour calculer la prédiction et l'augmentation de min_support réduit le nombre de voisins valides


Computing the msd similarity matrix...
Done computing similarity matrix.
Prédiction pour l'utilisateur 11 et l'élément 364 : 2.49203431372549
Utilisateur 11 a évalué l'élément 1214 avec une note de 3.17 (réel : 3.1333333333333333)
Utilisateur 11 a évalué l'élément 364 avec une note de 2.49 (réel : 3.1333333333333333)
Utilisateur 11 a évalué l'élément 4308 avec une note de 3.17 (réel : 3.1333333333333333)
Utilisateur 11 a évalué l'élément 527 avec une note de 3.90 (réel : 3.1333333333333333)
Utilisateur 13 a évalué l'élément 1997 avec une note de 2.80 (réel : 3.1333333333333333)
Utilisateur 13 a évalué l'élément 4993 avec une note de 2.80 (réel : 3.1333333333333333)
Utilisateur 13 a évalué l'élément 2700 avec une note de 2.80 (réel : 3.1333333333333333)
Utilisateur 13 a évalué l'élément 1721 avec une note de 2.80 (réel : 3.1333333333333333)
Utilisateur 13 a évalué l'élément 527 avec une note de 2.80 (réel : 3.1333333333333333)
Utilisateur 17 a évalué l'élément 2028 avec une note de 3.81

# 3. Implement and explore a customizable user-based algorithm
Create a self-made user-based algorithm allowing to customize the similarity metric, peer group calculation and aggregation function

In [73]:
class UserBased(AlgoBase):
    def __init__(self, k=3, min_k=1, sim_options={}, **kwargs):
        AlgoBase.__init__(self, sim_options=sim_options, **kwargs)
        self.k = k
        self.min_k = min_k

        
    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        # Calcul de la matrice des ratings
        self.compute_rating_matrix()
        # Calcul de la matrice de similarité
        self.compute_similarity_matrix()
        # Calcul de la moyenne des notes par utilisateur
        self.mean_ratings = []
        for u in range(self.trainset.n_users):
            user_ratings = []
            for (_, rating) in self.trainset.ur[u]: #_ correspond à l'index de l'item
                user_ratings.append(rating)
            if user_ratings:
                mean_rating = np.mean(user_ratings)
            else:
                mean_rating = float('nan')  # ou 0.0 si tu préfères éviter les NaN
            self.mean_ratings.append(mean_rating)

    
    def estimate(self, u, i):
            if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
                raise PredictionImpossible('User and/or item is unknown.')

            estimate = self.mean_ratings[u]

            ## Obtenir les voisins de l'utilisateur u qui ont noté l'item i
            neighbors = []
            for (v, rating) in self.trainset.ir[i]:  
                if v == u:
                    continue  # ne pas se comparer à soi-même

                sim_uv = self.sim[u, v]  # similarité entre u et v

                if sim_uv > 0 and not np.isnan(self.ratings_matrix[v, i]):  # si la similarité est positive et que v a noté l'item i
                    mean_v = self.mean_ratings[v]  # moyenne des notes de v
                    neighbors.append((sim_uv, rating - mean_v))

            # Trier les voisins par similarité décroissante
            neighbors.sort(key=lambda x: x[0], reverse=True)

            # Calcul de la moyenne pondérée sur les k meilleurs voisins
            actual_k = 0
            weighted_sum = 0.0
            sum_sim = 0.0

            for sim, rating_diff in neighbors:
                if actual_k == self.k:
                    break
                weighted_sum += sim * rating_diff
                sum_sim += sim
                actual_k += 1

            # Vérifier si on a suffisamment de voisins
            if actual_k >= self.min_k and sum_sim > 0:
                estimate += weighted_sum / sum_sim

            return estimate


                            
    def compute_rating_matrix(self):
        # -- implement here the compute_rating_matrix function --
        self.ratings_matrix = np.empty((self.trainset.n_users, self.trainset.n_items))
        self.ratings_matrix[:] = np.nan
        for u in range(self.trainset.n_users): # or each user
            for i, rating in self.trainset.ur[u]: #for each item rated by the user
                self.ratings_matrix[u, i] = rating

    
    def compute_similarity_matrix(self):
        m = self.trainset.n_users
        ratings_matrix = self.ratings_matrix
        min_support = self.sim_options.get('min_support', 1)
        sim_name = self.sim_options.get("name", "euclidean")  # valeur par défaut

        # Initialiser la matrice de similarité
        self.sim = np.eye(m)

        for i in range(m):
            for j in range(i + 1, m):  # j > i pour éviter les doublons
                row_i = ratings_matrix[i]
                row_j = ratings_matrix[j]

                if sim_name == "jacard":
                    sim = self.jaccard_similarity(row_i, row_j)
                    support = np.sum(~np.isnan(row_i) & ~np.isnan(row_j))
                else:
                    # Par défaut : similarité euclidienne normalisée
                    diff = row_i - row_j
                    support = np.sum(~np.isnan(diff))
                    if support >= min_support:
                        sim = 1 - np.nansum(diff ** 2) / support
                    else:
                        sim = 0

                if support >= min_support:
                    self.sim[i, j] = sim
                    self.sim[j, i] = sim

    def jaccard_similarity(self, row_i, row_j):
        # Masques binaires : True là où il y a une note
        mask_i = ~np.isnan(row_i)
        mask_j = ~np.isnan(row_j)

        intersection = np.sum(mask_i & mask_j)
        union = np.sum(mask_i | mask_j)

        if union == 0:
            return 0.0
        return intersection / union


# 4. Compare KNNWithMeans with UserBased
Try to replicate KNNWithMeans with your self-made UserBased and check that outcomes are identical

In [74]:
# -- assert that predictions are the same with different sim_options --
#comparaison entre requltat Kmeans et resultqts user based 
import surprise
from surprise import accuracy

# Paramètres de similarité
sim_options = {
    'name': 'msd',
    'user_based': True,
    'min_support': 3
}
k = 3
min_k = 2

# Ton algorithme custom
userbased_algo = UserBased(k=k, min_k=min_k, sim_options=sim_options)
userbased_algo.fit(trainset)

# Algo officiel de surprise
knn_algo = KNNWithMeans(k=k, min_k=min_k, sim_options=sim_options)
knn_algo.fit(trainset)

# Anti-testset (les notes absentes dans le trainset)
anti_testset = trainset.build_anti_testset()

# Comparer les 30 premières prédictions
print(f"{'UID':>5} {'IID':>5} {'KNN est.':>10} {'UserBased est.':>15} {'Diff':>10}")
print("-" * 50)

for i, (uid, iid, _) in enumerate(anti_testset[:30]):
    pred_knn = knn_algo.predict(uid, iid)
    pred_userbased = userbased_algo.predict(uid, iid)

    diff = abs(pred_knn.est - pred_userbased.est)

    print(f"{uid:>5} {iid:>5} {pred_knn.est:10.4f} {pred_userbased.est:15.4f} {diff:10.4f}")


Computing the msd similarity matrix...
Done computing similarity matrix.
  UID   IID   KNN est.  UserBased est.       Diff
--------------------------------------------------
   11  1214     3.1667          3.1667     0.0000
   11   364     2.4920          3.1667     0.6746
   11  4308     3.1667          3.1667     0.0000
   11   527     3.8989          3.1667     0.7322
   13  1997     2.8000          2.8000     0.0000
   13  4993     2.8000          2.8000     0.0000
   13  2700     2.8000          2.8000     0.0000
   13  1721     2.8000          2.8000     0.0000
   13   527     2.8000          2.8000     0.0000
   17  2028     3.8125          3.2500     0.5625
   17  4993     4.1283          3.2500     0.8783
   17  1214     3.2500          3.2500     0.0000
   17  4308     3.2500          3.2500     0.0000
   19  1997     3.5000          3.5000     0.0000
   19  2028     3.5000          3.5000     0.0000
   19  4993     3.5000          3.5000     0.0000
   19  5952     3.5000    

# 5. Compare MSD and Jacard
Compare predictions made with MSD similarity and Jacard similarity


In [ ]:
# -- compare predictions made with MSD similarity and Jacard similarity --
sim_options = {"name": "jacard", "min_support": 1}
algo = UserBased(k=3, min_k=1, sim_options=sim_options)
algo.fit(trainset)
predictions_jacard = algo.test(anti_testset)
print(f"Prédictions avec la similarité Jacard : {predictions_jacard[:30]}")


Prédictions avec la similarité Jacard : [Prediction(uid=11, iid=1214, r_ui=3.1333333333333333, est=2.7540272614622054, details={'was_impossible': False}), Prediction(uid=11, iid=364, r_ui=3.1333333333333333, est=2.8003300330033003, details={'was_impossible': False}), Prediction(uid=11, iid=4308, r_ui=3.1333333333333333, est=2.5907738095238093, details={'was_impossible': False}), Prediction(uid=11, iid=527, r_ui=3.1333333333333333, est=3.9878048780487805, details={'was_impossible': False}), Prediction(uid=13, iid=1997, r_ui=3.1333333333333333, est=2.8416666666666663, details={'was_impossible': False}), Prediction(uid=13, iid=4993, r_ui=3.1333333333333333, est=3.535763193052772, details={'was_impossible': False}), Prediction(uid=13, iid=2700, r_ui=3.1333333333333333, est=2.591666666666667, details={'was_impossible': False}), Prediction(uid=13, iid=1721, r_ui=3.1333333333333333, est=0.9691017316017314, details={'was_impossible': False}), Prediction(uid=13, iid=527, r_ui=3.1333333333333333

NameError: Wrong sim name jacard. Allowed values are cosine, msd, pearson, pearson_baseline.